In [ ]:
api_key = 'GYXJ7MUXI25UT0SZ'

In [ ]:
from IPython.core.display import HTML
import requests
import time
import pandas as pd
import datetime
HTML("""
<style>
.container { width:100% !important; }
</style>
""")

In [ ]:
def _get_data(symbols,time_from,time_to,api_key):
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbols}&time_from={time_from}&time_to={time_to}&limit=1000&apikey={api_key}"
    r = requests.get(url)
    data = r.json()
    return data

def _get_label_sentiment(x):
    if x <= -0.35:
        return 'Bearish','Bearish'
    elif -0.35 < x <= -0.15:
        return 'Somewhat-Bearish','Bearish'
    elif -0.15 < x < 0.15:
        return 'Neutral','Neutral'
    elif 0.15 <= x < 0.35:
        return 'Somewhat_Bullish','Bullish'
    else:  # x >= 0.35
        return 'Bullish','Bullish'

In [ ]:
def get_dataset(time_from="20030410T0130",
                time_to='',
                MAX_API_CALLS_PER_DAY = 25, # Free tier only allows 25 API calls per day
                MAX_API_CALLS_PER_MIN = 5 # Free tier only allows 5 api calls per minute
               ):
    data_list=[]
    for i in range(1,MAX_API_CALLS_PER_DAY+1):
        if i%5==0:
            time.sleep(60)

        data=_get_data('TSLA',time_from,time_to,api_key)
        if 'feed' not in data:break
        if len(data['feed'])==0: break
        data_list.append(data)
        time_to=data['feed'][-1]['time_published'][:-2] # Take all the way up to last 2 since api only takes minute level granularity
    df=pd.concat([pd.DataFrame(data['feed']) for data in data_list])
    # Extract TSLA specific relevance (we didn't use it in video)
    df['ticker_relevance_TSLA']=df['ticker_sentiment'].apply(lambda l:[el for el in l if el['ticker']=='TSLA'][0]['relevance_score']).astype(float)
    # Extract TSLA specific sentiment
    df['ticker_sentiment_TSLA']=df['ticker_sentiment'].apply(lambda l:[el for el in l if el['ticker']=='TSLA'][0]['ticker_sentiment_score']).astype(float)
    # Only take tickers with TSLA in headline
    df=df[df.title.str.contains('tsla|tesla',case=False)]
    # Extract # of tickers
    df['num_tickers']=df.ticker_sentiment.apply(lambda l:len(l))
    # Only take when # of tickers = 1
    df = df[df.num_tickers==1]
    # Applying the function and creating two new columns
    df[['detailed_original_label','label']] = df.apply(lambda row: _get_label_sentiment(row['ticker_sentiment_TSLA']), axis=1, result_type='expand')
    # Drop duplicates..
    df.drop_duplicates(subset=['summary'],inplace=True,keep='first')
    # Set index to time published
    df.set_index('time_published',inplace=True)
    # Sort by time published
    df.sort_index(inplace=True)
    return df

In [ ]:
df = get_dataset(time_to='')
df.to_csv('tsla_sentiment.csv')

In [ ]:
df

,title,url,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,ticker_relevance_TSLA,ticker_sentiment_TSLA,num_tickers,detailed_original_label,label
time_published,,,,,,,,,,,,,,,,,
20231227T224521,Tesla ( TSLA ) Exceeds Market Returns: Some ...,https://www.zacks.com/stock/news/2202845/tesla...,[Zacks Equity Research],Tesla (TSLA) reachead $261.47 at the closing o...,https://staticx-tuner.zacks.com/images/default...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Earnings', 'relevance_score': '0.9...",0.213644,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.5856...",0.585687,0.302055,1,Somewhat_Bullish,Bullish
20231228T023951,Elon Musk Draws Ire From Democratic Senators A...,https://www.benzinga.com/news/23/12/36406455/e...,[Benzinga Neuro],Two U.S. senators have written to Tesla Inc TS...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Manufacturing', 'relevance_score':...",-0.252409,Somewhat-Bearish,"[{'ticker': 'TSLA', 'relevance_score': '0.5990...",0.599092,-0.500821,1,Bearish,Bearish
20231228T032437,Elon Musk Says Tesla's Non-Beta FSD V12 Workin...,https://www.benzinga.com/news/23/12/36406568/e...,[Anan Ashraf],Tesla Inc TSLA CEO Elon Musk on Wednesday said...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Manufacturing', 'relevance_score':...",0.166800,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.5302...",0.530253,0.488797,1,Bullish,Bullish
20231228T050800,Elon Musk rips media reports of robot 'attack'...,https://www.businessinsider.com/elon-musk-slam...,[Kwan Wei Kevin Tan],Elon Musk slams media reports of robot 'attack...,None,Business Insider,GoogleRSS,www.businessinsider.com,"[{'topic': 'Manufacturing', 'relevance_score':...",-0.057931,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.5387...",0.538782,-0.322787,1,Somewhat-Bearish,Bearish
20231228T061110,Tesla CEO Elon Musk Slams Report Of Robot 'Att...,https://www.benzinga.com/news/23/12/36406867/t...,[Anan Ashraf],"Tesla Inc TSLA CEO Elon Musk, on Wednesday, ex...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.1...",-0.202214,Somewhat-Bearish,"[{'ticker': 'TSLA', 'relevance_score': '0.5313...",0.531326,-0.467332,1,Bearish,Bearish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20250321T064635,80 Tesla Vehicles Damaged At Dealership In Ham...,https://www.benzinga.com/news/global/25/03/444...,[Rounak Jain],"In another attack against Tesla Inc., TSLA Ham...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Manufacturing', 'relevance_score':...",-0.146659,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.8040...",0.804025,-0.586211,1,Bearish,Bearish
20250321T095328,Tesla Owners Might Have To Pay More For Insura...,https://www.benzinga.com/markets/equities/25/0...,[Rounak Jain],Insurance premiums for Tesla Inc. TSLA vehicle...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.1...",0.009618,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.8468...",0.846873,-0.151219,1,Somewhat-Bearish,Bearish
20250321T123938,Tesla Record Highest Trade-Ins Amid Backlash A...,https://www.benzinga.com/markets/equities/25/0...,[Rounak Jain],A record number of Tesla Inc. TSLA vehicles ha...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.038839,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.8040...",0.804025,-0.133946,1,Neutral,Neutral


In [ ]:
df.shape


(2307, 17)

In [ ]:
df.describe

<bound method NDFrame.describe of                                                              title  \
time_published                                                       
20231227T224521  Tesla  ( TSLA )  Exceeds Market Returns: Some ...   
20231228T023951  Elon Musk Draws Ire From Democratic Senators A...   
20231228T032437  Elon Musk Says Tesla's Non-Beta FSD V12 Workin...   
20231228T050800  Elon Musk rips media reports of robot 'attack'...   
20231228T061110  Tesla CEO Elon Musk Slams Report Of Robot 'Att...   
...                                                            ...   
20250321T064635  80 Tesla Vehicles Damaged At Dealership In Ham...   
20250321T095328  Tesla Owners Might Have To Pay More For Insura...   
20250321T123938  Tesla Record Highest Trade-Ins Amid Backlash A...   
20250321T132300  1 Wall Street Analyst Thinks Tesla Will Hit $4...   
20250321T140903  SpaceX Dethrones Tesla As Elon Musk's Most Val...   

                                                               url  \
time_published                                                       
20231227T224521  https://www.zacks.com/stock/news/2202845/tesla...   
20231228T023951  https://www.benzinga.com/news/23/12/36406455/e...   
20231228T032437  https://www.benzinga.com/news/23/12/36406568/e...   
20231228T050800  https://www.businessinsider.com/elon-musk-slam...   
20231228T061110  https://www.benzinga.com/news/23/12/36406867/t...   
...                                                            ...   
20250321T064635  https://www.benzinga.com/news/global/25/03/444...   
20250321T095328  https://www.benzinga.com/markets/equities/25/0...   
20250321T123938  https://www.benzinga.com/markets/equities/25/0...   
20250321T132300  https://www.fool.com/investing/2025/03/21/1-wa...   
20250321T140903  https://www.benzinga.com/markets/25/03/4443694...   

                                 authors  \
time_published                             
20231227T224521  [Zacks Equity Research]   
20231228T023951         [Benzinga Neuro]   
20231228T032437            [Anan Ashraf]   
20231228T050800     [Kwan Wei Kevin Tan]   
20231228T061110            [Anan Ashraf]   
...                                  ...   
20250321T064635            [Rounak Jain]   
20250321T095328            [Rounak Jain]   
20250321T123938            [Rounak Jain]   
20250321T132300          [Jeremy Bowman]   
20250321T140903            [Namrata Sen]   

                                                           summary  \
time_published                                                       
20231227T224521  Tesla (TSLA) reachead $261.47 at the closing o...   
20231228T023951  Two U.S. senators have written to Tesla Inc TS...   
20231228T032437  Tesla Inc TSLA CEO Elon Musk on Wednesday said...   
20231228T050800  Elon Musk slams media reports of robot 'attack...   
20231228T061110  Tesla Inc TSLA CEO Elon Musk, on Wednesday, ex...   
...                                                            ...   
20250321T064635  In another attack against Tesla Inc., TSLA Ham...   
20250321T095328  Insurance premiums for Tesla Inc. TSLA vehicle...   
20250321T123938  A record number of Tesla Inc. TSLA vehicles ha...   
20250321T132300  Tesla ( NASDAQ: TSLA ) has long been a battleg...   
20250321T140903  Elon Musk's SpaceX has surpassed Tesla Inc. TS...   

                                                      banner_image  \
time_published                                                       
20231227T224521  https://staticx-tuner.zacks.com/images/default...   
20231228T023951  https://cdn.benzinga.com/files/images/story/20...   
20231228T032437  https://cdn.benzinga.com/files/images/story/20...   
20231228T050800                                               None   
20231228T061110  https://cdn.benzinga.com/files/images/story/20...   
...                                                            ...   
20250321T064635  https://cdn.benzinga.com/files/images/story/20...   
20250321T095328  https://cdn.b

In [ ]:
dataset = pd.read_csv("/content/tsla_sentiment-3.csv")

In [ ]:
dataset

,time_published,title,url,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,ticker_relevance_TSLA,ticker_sentiment_TSLA,num_tickers,detailed_original_label,label
0,20231227T224521,Tesla ( TSLA ) Exceeds Market Returns: Some ...,https://www.zacks.com/stock/news/2202845/tesla...,['Zacks Equity Research'],Tesla (TSLA) reachead $261.47 at the closing o...,https://staticx-tuner.zacks.com/images/default...,Zacks Commentary,NaN,www.zacks.com,"[{'topic': 'Earnings', 'relevance_score': '0.9...",0.213644,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.5856...",0.585687,0.302055,1,Somewhat_Bullish,Bullish
1,20231228T023951,Elon Musk Draws Ire From Democratic Senators A...,https://www.benzinga.com/news/23/12/36406455/e...,['Benzinga Neuro'],Two U.S. senators have written to Tesla Inc TS...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Manufacturing', 'relevance_score':...",-0.252409,Somewhat-Bearish,"[{'ticker': 'TSLA', 'relevance_score': '0.5990...",0.599092,-0.500821,1,Bearish,Bearish
2,20231228T032437,Elon Musk Says Tesla's Non-Beta FSD V12 Workin...,https://www.benzinga.com/news/23/12/36406568/e...,['Anan Ashraf'],Tesla Inc TSLA CEO Elon Musk on Wednesday said...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Manufacturing', 'relevance_score':...",0.166800,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.5302...",0.530253,0.488797,1,Bullish,Bullish
3,20231228T050800,Elon Musk rips media reports of robot 'attack'...,https://www.businessinsider.com/elon-musk-slam...,['Kwan Wei Kevin Tan'],Elon Musk slams media reports of robot 'attack...,NaN,Business Insider,GoogleRSS,www.businessinsider.com,"[{'topic': 'Manufacturing', 'relevance_score':...",-0.057931,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.5387...",0.538782,-0.322787,1,Somewhat-Bearish,Bearish
4,20231228T061110,Tesla CEO Elon Musk Slams Report Of Robot 'Att...,https://www.benzinga.com/news/23/12/36406867/t...,['Anan Ashraf'],"Tesla Inc TSLA CEO Elon Musk, on Wednesday, ex...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.1...",-0.202214,Somewhat-Bearish,"[{'ticker': 'TSLA', 'relevance_score': '0.5313...",0.531326,-0.467332,1,Bearish,Bearish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2302,20250321T064635,80 Tesla Vehicles Damaged At Dealership In Ham...,https://www.benzinga.com/news/global/25/03/444...,['Rounak Jain'],"In another attack against Tesla Inc., TSLA Ham...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Manufacturing', 'relevance_score':...",-0.146659,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.8040...",0.804025,-0.586211,1,Bearish,Bearish
2303,20250321T095328,Tesla Owners Might Have To Pay More For Insura...,https://www.benzinga.com/markets/equities/25/0...,['Rounak Jain'],Insurance premiums for Tesla Inc. TSLA vehicle...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.1...",0.009618,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.8468...",0.846873,-0.151219,1,Somewhat-Bearish,Bearish
2304,20250321T123938,Tesla Record Highest Trade-Ins Amid Backlash A...,https://www.benzinga.com/markets/equities/25/0...,['Rounak Jain'],A record number of Tesla Inc. TSLA vehicles ha...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.038839,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.8040...",0.804025,-0.133946,1,Neutral,Neutral
2305,20250321T132300,1 Wall Street Analyst Thinks Tesla Will Hit $4...,https://www.fool.com/investing/2025/03/21/1-wa...,['Jeremy Bowman'],Tesla ( NASDAQ: TSLA ) has long been a battleg...,https://g.f

In [ ]:
dataset.isnull().sum()

,0
time_published,0
title,0
url,0
authors,0
summary,0
banner_image,244
source,0
category_within_source,464
source_domain,0
topics,0


In [ ]:
df

,title,url,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,ticker_relevance_TSLA,ticker_sentiment_TSLA,num_tickers,detailed_original_label,label
time_published,,,,,,,,,,,,,,,,,
20230209T200727,Tesla shares rebound from January bottom,https://www.foxbusiness.com/markets/tesla-shar...,[],Shares of Tesla cleared $200 on Thursday as th...,https://cf-images.us-east-1.prod.boltdns.net/v...,Fox Business News,n/a,www.foxbusiness.com,"[{'topic': 'Earnings', 'relevance_score': '0.8...",0.189360,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.4563...",0.456340,-0.009141,1,Neutral,Neutral
20230210T015100,Tesla hikes price of Model Y rear wheel drive ...,https://www.reuters.com/business/autos-transpo...,[Reuters],[1/2] Model Y cars are pictured during the ope...,https://cloudfront-us-east-2.images.arcpublish...,Reuters,Business,www.reuters.com,"[{'topic': 'Manufacturing', 'relevance_score':...",0.078331,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.8760...",0.876064,0.138092,1,Neutral,Neutral
20230210T033042,Tesla Pushes Up Model Y Prices In China - Tesl...,https://www.benzinga.com/markets/asia/23/02/30...,[Shivdeep Dhaliwal],Tesla Inc TSLA is raising the prices of its Mo...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.1...",-0.085149,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.7370...",0.737032,-0.027559,1,Neutral,Neutral
20230210T113016,Tesla does a flip-flop by lifting Model Y car ...,https://africa.businessinsider.com/markets/tes...,[],Tesla does a flip-flop by lifting Model Y car ...,https://ocdn.eu/pulscms-transforms/1/HfiktkuTU...,Business Insider,GoogleRSS,africa.businessinsider.com,"[{'topic': 'Manufacturing', 'relevance_score':...",0.180567,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.5742...",0.574235,0.340140,1,Somewhat_Bullish,Bullish
20230210T113016,Tesla does a flip-flop by lifting Model Y car ...,https://markets.businessinsider.com/news/stock...,[Ryan Hogg],Tesla Flip-Flops by Lifting Model Y Prices in ...,https://i.insider.com/63923e758580f70019f812c3...,Business Insider,GoogleRSS,markets.businessinsider.com,"[{'topic': 'Manufacturing', 'relevance_score':...",0.101991,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.7246...",0.724687,0.150626,1,Somewhat_Bullish,Bullish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20250321T064635,80 Tesla Vehicles Damaged At Dealership In Ham...,https://www.benzinga.com/news/global/25/03/444...,['Rounak Jain'],"In another attack against Tesla Inc., TSLA Ham...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Manufacturing', 'relevance_score':...",-0.146659,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.8040...",0.804025,-0.586211,1,Bearish,Bearish
20250321T095328,Tesla Owners Might Have To Pay More For Insura...,https://www.benzinga.com/markets/equities/25/0...,['Rounak Jain'],Insurance premiums for Tesla Inc. TSLA vehicle...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.1...",0.009618,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.8468...",0.846873,-0.151219,1,Somewhat-Bearish,Bearish
20250321T123938,Tesla Record Highest Trade-Ins Amid Backlash A...,https://www.benzinga.com/markets/equities/25/0...,['Rounak Jain'],A record number of Tesla Inc. TSLA vehicles ha...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.038839,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.8040...",0.804025,-0.133946,1,Neutral,Neutral


In [ ]:
data = get_dataset(time_to=earliest_time_published[:-2])

In [ ]:
df = pd.concat([data3,df])
df.drop_duplicates(subset=['summary'],inplace=True,keep='first')
df.to_csv('tsla_sentiment.csv')

In [ ]:
df

,title,url,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,ticker_relevance_TSLA,ticker_sentiment_TSLA,num_tickers,detailed_original_label,label
time_published,,,,,,,,,,,,,,,,,
20230129T061700,US investigates self-driving claims made by El...,https://www.business-standard.com/article/inte...,[IANS],US investigates self-driving claims made by El...,https://bsmedia.business-standard.com/_media/b...,Business Standard,GoogleRSS,www.business-standard.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.052691,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.8749...",0.874993,0.145067,1,Neutral,Neutral
20230129T114500,Why 2023 Is Tesla's Year To Prove Itself,https://www.fool.com/investing/2023/01/29/tesl...,[Travis Hoium and Jason Hall],Tesla's fourth-quarter report showed both the ...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Earnings', 'relevance_score': '0.3...",0.073280,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.9850...",0.985002,0.175709,1,Somewhat_Bullish,Bullish
20230129T124300,Elon Musk Finally Reveals Sales Increase Follo...,https://www.fool.com/investing/2023/01/29/elon...,"[CFA, Parkev Tatevosian]",There was much speculation among investors on ...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Manufacturing', 'relevance_score':...",0.143396,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.9470...",0.947069,0.367920,1,Bullish,Bullish
20230129T140857,Tesla 'spontaneously' catches fire on Californ...,https://www.foxbusiness.com/technology/tesla-s...,[],"A Tesla Model S ""spontaneously"" burst into fla...",https://static.foxbusiness.com/foxbusiness.com...,Fox Business News,n/a,www.foxbusiness.com,"[{'topic': 'Manufacturing', 'relevance_score':...",-0.183315,Somewhat-Bearish,"[{'ticker': 'TSLA', 'relevance_score': '0.5742...",0.574235,0.164179,1,Somewhat_Bullish,Bullish
20230129T211841,Would You Sell Your House For Tesla Stock? Thi...,https://www.benzinga.com/news/23/01/30615938/w...,[Chris Katje],Last year marked Tesla Inc's TSLA worst stock ...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.8...",0.246707,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.9737...",0.973732,0.434486,1,Bullish,Bullish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20250321T064635,80 Tesla Vehicles Damaged At Dealership In Ham...,https://www.benzinga.com/news/global/25/03/444...,['Rounak Jain'],"In another attack against Tesla Inc., TSLA Ham...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Manufacturing', 'relevance_score':...",-0.146659,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.8040...",0.804025,-0.586211,1,Bearish,Bearish
20250321T095328,Tesla Owners Might Have To Pay More For Insura...,https://www.benzinga.com/markets/equities/25/0...,['Rounak Jain'],Insurance premiums for Tesla Inc. TSLA vehicle...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.1...",0.009618,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.8468...",0.846873,-0.151219,1,Somewhat-Bearish,Bearish
20250321T123938,Tesla Record Highest Trade-Ins Amid Backlash A...,https://www.benzinga.com/markets/equities/25/0...,['Rounak Jain'],A record number of Tesla Inc. TSLA vehicles ha...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.038839,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.8040...",0.804025,-0.133946,1,Neutral,Neutral


In [ ]:
df.keys()

Index(['title', 'url', 'authors', 'summary', 'banner_image', 'source',
       'category_within_source', 'source_domain', 'topics',
       'overall_sentiment_score', 'overall_sentiment_label',
       'ticker_sentiment', 'ticker_relevance_TSLA', 'ticker_sentiment_TSLA',
       'num_tickers', 'detailed_original_label', 'label'],
      dtype='object')